In [73]:
import pandas as pd
import matplotlib
from io import StringIO

# 1) Load the CSV data, set "Country" as index, and convert to decimal fractions:
csv_data = """
Country,2009,2010,2011,2012,2013,2015,2016,2017,2018,2019,2020,2021
Austria,1.025,1.040,1.215,1.449,1.881,0.878,0.762,1.954,0.952,0.650,1.289,0.286
Belgium,1.083,1.119,2.768,1.991,1.737,1.500,1.235,0.762,1.138,1.461,1.529,1.344
Denmark,1.890,1.690,1.467,1.817,1.702,1.089,1.085,0.662,0.604,0.595,0.692,0.501
France,1.109,1.017,0.808,1.366,1.248,0.587,0.721,0.583,0.767,1.086,0.880,0.635
Germany,1.421,1.197,1.235,1.468,1.663,1.339,1.426,1.003,1.011,0.838,0.752,0.445
Ireland,7.786,10.142,16.884,16.988,10.462,2.918,2.185,1.996,2.839,3.821,1.878,1.989
Italy,3.240,2.134,3.474,5.212,6.838,3.571,1.877,6.903,5.374,1.378,0.687,0.670
Luxembourg,8.130,1.049,1.445,5.409,7.379,0.599,1.665,0.784,0.483,0.416,0.992,0.468
Netherlands,1.611,2.174,1.787,2.502,3.752,2.315,1.547,2.322,1.700,1.734,1.530,0.888
Norway,0.878,0.986,1.013,1.395,0.520,0.950,1.452,1.447,1.468,1.220,1.823,1.448
Poland,5.379,3.797,2.608,3.643,3.183,0.898,0.980,1.147,0.859,1.523,0.791,0.962
Portugal,2.598,2.005,5.247,7.104,5.975,11.790,5.881,3.115,2.743,1.598,1.511,2.083
Spain,5.004,4.042,3.539,6.697,8.560,5.234,3.827,3.184,2.062,1.824,1.380,3.118
Sweden,0.678,0.454,0.181,0.338,0.342,0.303,0.257,0.177,0.168,0.332,0.250,0.065
United Kingdom,1.423,1.071,2.509,1.996,2.379,0.768,1.025,1.237,0.887,1.267,1.165,1.384
"""
df = pd.read_csv(StringIO(csv_data)).set_index('Country')
df /= 100  # Convert percentages to decimal (e.g., 1.025% -> 0.01025)

# 2) Reset index to make "Country" a normal column
df_reset = df.reset_index()

# 3) Create a row of average values
numeric_cols = df_reset.select_dtypes(include='number').columns
mean_vals = df_reset[numeric_cols].mean()
avg_row = pd.DataFrame([["Average"] + list(mean_vals)], columns=df_reset.columns)
df_with_avg = pd.concat([df_reset, avg_row], ignore_index=True)

# 4) Compute per-row "Max DF" and "Mean DF"
df_with_avg["Max DF"] = df_with_avg[numeric_cols].max(axis=1)
df_with_avg["Mean DF"] = df_with_avg[numeric_cols].mean(axis=1)

# Update numeric_cols (since we added two new columns)
numeric_cols = df_with_avg.select_dtypes(include='number').columns

# 5) Define a color gradient function for each row
def highlight_row(row):
    vals = row[numeric_cols]
    norm = matplotlib.colors.TwoSlopeNorm(
        vcenter=vals.mean(),
        vmin=vals.min(),
        vmax=vals.max()
    )
    cmap = matplotlib.colormaps.get_cmap('RdYlGn_r')
    styles = []
    for col in row.index:
        if col in numeric_cols:
            r, g, b, _ = cmap(norm(row[col]))
            styles.append(f'background-color: {matplotlib.colors.to_hex((r, g, b, 0.4), keep_alpha=True)}')
        else:
            styles.append('')
    return styles

# 6) Apply styling
styled = (
    df_with_avg
    .style
    .format("{:.2%}", subset=numeric_cols)  # Format numeric columns as % with 2 decimals
    .apply(highlight_row, axis=1)          # Apply gradient row-wise
    .set_properties(subset=numeric_cols, **{"color": "black"})
    .set_table_styles([
        {"selector": "table",                   "props": [("border-collapse", "collapse"), ("border", "none")]},
        {"selector": "th",                      "props": [("border", "none")]},
        {"selector": "td",                      "props": [("border", "none")]},
        {"selector": "tbody tr:last-child td",  "props": [("font-weight", "bold")]}  # Make last row bold
    ])
)

styled


,Country,2009,2010,2011,2012,2013,2015,2016,2017,2018,2019,2020,2021,Max DF,Mean DF
0,Austria,1.02%,1.04%,1.22%,1.45%,1.88%,0.88%,0.76%,1.95%,0.95%,0.65%,1.29%,0.29%,1.95%,1.12%
1,Belgium,1.08%,1.12%,2.77%,1.99%,1.74%,1.50%,1.24%,0.76%,1.14%,1.46%,1.53%,1.34%,2.77%,1.47%
2,Denmark,1.89%,1.69%,1.47%,1.82%,1.70%,1.09%,1.08%,0.66%,0.60%,0.59%,0.69%,0.50%,1.89%,1.15%
3,France,1.11%,1.02%,0.81%,1.37%,1.25%,0.59%,0.72%,0.58%,0.77%,1.09%,0.88%,0.64%,1.37%,0.90%
4,Germany,1.42%,1.20%,1.24%,1.47%,1.66%,1.34%,1.43%,1.00%,1.01%,0.84%,0.75%,0.45%,1.66%,1.15%
5,Ireland,7.79%,10.14%,16.88%,16.99%,10.46%,2.92%,2.19%,2.00%,2.84%,3.82%,1.88%,1.99%,16.99%,6.66%
6,Italy,3.24%,2.13%,3.47%,5.21%,6.84%,3.57%,1.88%,6.90%,5.37%,1.38%,0.69%,0.67%,6.90%,3.45%
7,Luxembourg,8.13%,1.05%,1.45%,5.41%,7.38%,0.60%,1.67%,0.78%,0.48%,0.42%,0.99%,0.47%,8.13%,2.40%
8,Netherlands,1.61%,2.17%,1.79%,2.50%,3.75%,2.31%,1.55%,2.32%,1.70%,1.73%,1.53%,0.89%,3.75%,1.99%
9,Norway,0.88%,0.99%,1.01%,1.40%,0.52%,0.95%,1.45%,1.45%,1.47%,1.22%,1.82%,1.45%,1.82%,1.22%
